In [ ]:
import numpy as np
import teneva
from time import perf_counter as tpc
np.random.seed(42)

In [ ]:
np.random.seed(42)
# Target function:
f = lambda x: 10. - np.sum(x**2)
f_batch = lambda X: np.array([f(x) for x in X])

d = 5                              # Dimension
a = [-2.]*d                        # Grid lower bounds
b = [+2.]*d                        # Grid upper bounds
n = [201]*d                        # Grid size

In [ ]:
a, b

In [ ]:
# We build very accurate approximation of the function:
Y0 = teneva.rand(n, r=2, seed=42)  # Initial approximation for TT-cross
Y = teneva.cross(lambda I: f_batch(teneva.ind_to_poi(I, a, b, n, 'cheb')),
    Y0, m=5.E+5, e=None, log=True)
Y = teneva.truncate(Y, 1.E-16)

# We compute the TT-tensor for Chebyshev interpolation coefficients:
A = teneva.func_int(Y)

In [ ]:
# We find the maximum modulo point:
x_opt = teneva.optima_func_tt_beam(A, k=10)     
y_opt = teneva.func_get(x_opt, A, a, b)

print(f'x opt appr :', x_opt)
print(f'y opt appr :', y_opt)

In [ ]:
?teneva.optima_func_tt_beam

In [ ]:
?teneva.ind_to_poi

In [ ]:
"""The demo of using ttopt for maximization. Example with QTT.

We'll find the maximum for the 10-dimensional Alpine function with vectorized
input. The target function for maximization has the form f(X), where input X is
the [samples, dimension] numpy array.

Run it from the root of the project as "python demo/qtt_max.py".

As a result of the script work we expect the output in console like this:
"
...
Alpine-10d | evals=1.00e+05 | t_cur=1.65e-01 | y= 8.715206e+01
----------------------------------------------------------------------
Alpine-10d | evals=1.00e+05 | t_all=2.22e+00 | y= 8.715206e+01 
"

"""
import numpy as np


from ttopt import TTOpt
from ttopt import ttopt_init


np.random.seed(42)


d = 10                     # Number of function dimensions:
rank = 4                    # Maximum TT-rank while cross-like iterations
#def f(X):                   # Target function
#    return np.sum(np.abs(X * np.sin(X) + 0.1 * X), axis=1)

f = lambda X: (10. - np.sum(X**2, axis = 1))

# We initialize the TTOpt class instance with the correct parameters:
tto = TTOpt(
    f=f,                    # Function for maximization. X is [samples, dim]
    d=d,                    # Number of function dimensions
    a=-10.,                 # Grid lower bound (number or list of len d)
    b=+10.,                 # Grid upper bound (number or list of len d)
    p=2,                    # The grid size factor (there will n=p^q points)
    q=20,                   # The grid size factor (there will n=p^q points)
    evals=5.E+6,            # Number of function evaluations
    name='Alpine',          # Function name for log (this is optional)
    with_log=True)


# And now we launching the maximizer:
tto.optimize(rank, is_max=True)


# We can extract the results of the computation:
x = tto.x_opt          # The found value of the maximum of the function (x)
y = tto.y_opt          # The found value of the maximum of the function (y=f(x))
k_c = tto.k_cache      # Total number of cache usage (should be 0 in this demo)
k_e = tto.k_evals      # Total number of requests to func (is always = evals)
k_t = tto.k_total      # Total number of requests (k_cache + k_evals)
t_f = tto.t_evals_mean # Average time spent to real function call for 1 point
                       # ... (see "ttopt.py" and docs for more details)


# We log the final state:
print('-' * 70 + '\n' + tto.info() +'\n\n')

In [ ]:
tto.x_opt

In [ ]:
?TTOpt

In [ ]:
import numpy as np
import copy
from gmpy2 import mpfr, get_context
get_context().precision=400


# Import of functions for creation of GP from the data, 
# separate training
#thhermodynamic consts
from GPPhad import create_from_scratch, retrain, consts

# Import of covariance functions, 
#class for GP, 
#printing function
# function for trained GP loading
from GPPhad import cov_real, GP_full, print_point, load

In [ ]:
th_full = [mpfr('-6.247035679999999757683326606638729572296142578125',400),
 mpfr('0.292807738541853712632700990070588886737823486328125',400),
 mpfr('0.456572889549176519352613468072377145290374755859375',400),
 mpfr('-0.04191095261276779615489118668847368098795413970947265625',400),
 mpfr('0.05475806710280016476044551154700457118451595306396484375',400),
 mpfr('0.10669002480846336011754971195841790176928043365478515625',400),
 mpfr('-5.05760411554052335958431285689584910869598388671875',400),
 mpfr('-0.88172228690680132245915956445969641208648681640625',400),
 mpfr('0.0298605271304240964258536195075066643767058849334716796875',400),
 mpfr('-0.007369150920216244009253170332840454648248851299285888671875',400),
 mpfr('11.3855892321983223069992163800634443759918212890625',400),
 mpfr('-3.817151634735199028369834195473231375217437744140625',400),
 mpfr('0.27150991454327677576685573512804694473743438720703125',400),
 mpfr('0.56436708068229746171340366345248185098171234130859375',400),
 mpfr('-0.06247883283127071696316789939373848028481006622314453125',400),
 mpfr('-0.035707238514284513064556136896499083377420902252197265625',400),
 mpfr('-1.2659861741798399403791108852601610124111175537109375',400)]

In [ ]:
phases = ['liq', 'sol_fcc', 'sol_bcc']
x_fixed = [5, 10**30]

GP_Li = create_from_scratch(cov_real, th_full, phases, x_fixed = x_fixed)

In [ ]:
GP_Li.marg_like(th_full, recomp = False)

In [ ]:
# Bounds for phases volumes
GP_Li.bounds = {"liq": [10, 24], "sol_fcc": [10, 15], 'sol_bcc': [10, 24]}

# Melting points with error
melt_points = [[['sol_fcc', 'liq'], [12/consts['Pk'], 495.5*consts['k'], 1.6*consts['k']]],
                [['sol_bcc', 'liq'], [0/consts['Pk'], 475.7*consts['k'], 1.7*consts['k']]]]

# Index of hyperparameters that correspond to phases
ind_bounds = {'liq': range(0, 6), 'sol_fcc': range(6, 11), 'sol_bcc': range(11, 17)}

# Separate train of phases
GP_Li, th_temp = retrain(GP_Li, melt_points, ind_bounds)

In [ ]:
GP_Li.th

In [ ]:
%timeit GP_Li.marg_like(th_temp, recomp = False)

In [237]:
import numpy as np
from scipy.optimize import minimize
import gmpy2 as gp
gp.get_context().precision=400

import mpinv
from mpinv import fast_mp_matrix_inverse_symm as f_mp_inv
from mpinv import fast_mp_matrix_logdet_symm as f_mp_logdet

def marg_like(self, th, phase = None, recomp = False):
    """Marginal likelihood estimation for the given Gaussian process and list of hyperparameters

    INPUT:

    set of hyperparameters (th)

    phase (phase)

    optinal recomputation of inverve covariance matrix of the GP (recomp)

    OUTPUT:

    -mar - marginal likelihood value * -1

    """
    if not self.melt:
        if recomp:
            print(phase, th)
            self.th = th

            # Consruction of new covarience matrix (K) and its inverse (K_inv) for particular phase
            self.K[phase] = self.constr_matrix(self.X[phase])
            self.err_m[phase] = self.constr_matrix(self.err[phase])
            self.K_inv[phase] = f_mp_inv(self.K[phase] + self.err_m[phase])

        # [0, 0] element corresponds to the value
        val_1 = 1./2.*(self.Y[phase].T@self.K_inv[phase]@self.Y[phase])[0, 0]
        val_2 = 1./2.*f_mp_logdet(self.K[phase] + self.err_m[phase])
        val_3 = len(self.Y[phase])/2.*gp.log(2.*gp.const_pi(200))

        mar = -val_1 - val_2 - val_3
        print(-mar)
    else:
        if recomp:
            self.th = th

            # Consruction of new covarience matrix (K) and its inverse (K_inv) for particular phase
            K = np.array(self.constr_matrix_melt(self.X), dtype = np.float64)
            err_m = np.array(self.constr_matrix_melt(self.err), dtype = np.float64)
            #K_inv = np.linalg.inv(K + err_m)
        
            L = np.linalg.cholesky(K + err_m)
            #L_inv = np.linalg.inv(L)
            
            #K_inv =  L_inv.T @ L_inv
            
            Y = np.array(self.Y, dtype = np.float64)
            
            alpha = np.linalg.solve(L.T, np.linalg.solve(L, Y))
            #alpha = 
            
            #print(alpha.T.shape)
            #val_1 = 1./2.*(Y.T@K_inv@Y)[0, 0] # [0, 0] element corresponds to the value
            #val_2 = 1./2.*f_mp_logdet(K + err_m)
            val_1 = 1/2.* Y[:, 0] @ alpha[:, 0]
            #val_1 = 0.
            val_2 =  np.sum(np.log(np.diag(L)))
            
            val_3 = len(self.Y)/2.*np.log(2.*np.pi)
            
            
        mar = -val_1 - val_2 - val_3
        #print(-mar)

    if 0.0 == np.nan_to_num(np.float64(-mar)):
        return 10**10
    else:
        return -np.float64(-mar)

In [238]:
GP_Li.Y.shape

(154, 1)

In [239]:
%timeit marg_like(GP_Li, th_temp_arr, recomp = True)

194 ms ± 1.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [240]:
marg_like(GP_Li, th_temp_arr, recomp = True)

291.4834812797729

In [241]:
marg_like_fit = lambda th: marg_like(GP_Li, th, recomp = True)

th_temp_arr = np.array(th_temp, dtype = np.float64)
th_low = (th_temp_arr - np.abs(th_temp_arr) * 0.2).tolist()
th_up = (th_temp_arr + np.abs(th_temp_arr) * 0.2).tolist()

In [230]:
marg_like(GP_Li, th_up, recomp = True), marg_like(GP_Li, th_low, recomp = True)

(267.0305543202149, 209.30247004412115)

In [231]:
marg_like_fit(th_temp_arr)

297.4557572251796

In [232]:
th_temp_arr.shape

(17,)

In [234]:
"""The demo of using ttopt for maximization. Example with QTT.

We'll find the maximum for the 10-dimensional Alpine function with vectorized
input. The target function for maximization has the form f(X), where input X is
the [samples, dimension] numpy array.

Run it from the root of the project as "python demo/qtt_max.py".

As a result of the script work we expect the output in console like this:
"
...
Alpine-10d | evals=1.00e+05 | t_cur=1.65e-01 | y= 8.715206e+01
----------------------------------------------------------------------
Alpine-10d | evals=1.00e+05 | t_all=2.22e+00 | y= 8.715206e+01 
"

"""
import numpy as np


from ttopt import TTOpt
from ttopt import ttopt_init


np.random.seed(42)


d = 17                    # Number of function dimensions:
rank = 4                    # Maximum TT-rank while cross-like iterations
#def f(X):                   # Target function
#    return np.sum(np.abs(X * np.sin(X) + 0.1 * X), axis=1)

f = lambda X: (10. - np.sum(X**2, axis = 1))

# We initialize the TTOpt class instance with the correct parameters:
tto = TTOpt(
    f=marg_like_fit,                    # Function for maximization. X is [samples, dim]
    d=d,                    # Number of function dimensions
    a=th_low,                 # Grid lower bound (number or list of len d)
    b=th_up,                 # Grid upper bound (number or list of len d)
    p=4,                    # The grid size factor (there will n=p^q points)
    q=30,                   # The grid size factor (there will n=p^q points)
    evals=1.E+2,            # Number of function evaluations
    name='Alpine',          # Function name for log (this is optional)
    with_log=True,
    is_vect = False)


# And now we launching the maximizer:
tto.optimize(rank, is_max=True)


# We can extract the results of the computation:
x = tto.x_opt          # The found value of the maximum of the function (x)
y = tto.y_opt          # The found value of the maximum of the function (y=f(x))
k_c = tto.k_cache      # Total number of cache usage (should be 0 in this demo)
k_e = tto.k_evals      # Total number of requests to func (is always = evals)
k_t = tto.k_total      # Total number of requests (k_cache + k_evals)
t_f = tto.t_evals_mean # Average time spent to real function call for 1 point
                       # ... (see "ttopt.py" and docs for more details)


# We log the final state:
print('-' * 70 + '\n' + tto.info() +'\n\n')


Alpine-17d | evals=8.00e+01 | t_cur=1.55e+01 | y= 2.914835e+02 
Alpine-17d | evals=1.00e+02 | t_cur=1.93e+01 | y= 2.914835e+02 
----------------------------------------------------------------------
Alpine-17d | evals=1.00e+02 | t_all=1.94e+01 | y= 2.914835e+02 




In [235]:
tto.x_opt

th_temp_arr = np.array(tto.x_opt, dtype = np.float64)
th_low = (th_temp_arr - np.abs(th_temp_arr) * 0.4).tolist()
th_up = (th_temp_arr + np.abs(th_temp_arr) * 0.4).tolist()

In [236]:
"""The demo of using ttopt for maximization. Example with QTT.

We'll find the maximum for the 10-dimensional Alpine function with vectorized
input. The target function for maximization has the form f(X), where input X is
the [samples, dimension] numpy array.

Run it from the root of the project as "python demo/qtt_max.py".

As a result of the script work we expect the output in console like this:
"
...
Alpine-10d | evals=1.00e+05 | t_cur=1.65e-01 | y= 8.715206e+01
----------------------------------------------------------------------
Alpine-10d | evals=1.00e+05 | t_all=2.22e+00 | y= 8.715206e+01 
"

"""
import numpy as np


from ttopt import TTOpt
from ttopt import ttopt_init


np.random.seed(42)


d = 17                    # Number of function dimensions:
rank = 4                    # Maximum TT-rank while cross-like iterations
#def f(X):                   # Target function
#    return np.sum(np.abs(X * np.sin(X) + 0.1 * X), axis=1)

f = lambda X: (10. - np.sum(X**2, axis = 1))

# We initialize the TTOpt class instance with the correct parameters:
tto = TTOpt(
    f=marg_like_fit,                    # Function for maximization. X is [samples, dim]
    d=d,                    # Number of function dimensions
    a=th_low,                 # Grid lower bound (number or list of len d)
    b=th_up,                 # Grid upper bound (number or list of len d)
    p=2,                    # The grid size factor (there will n=p^q points)
    q=30,                   # The grid size factor (there will n=p^q points)
    evals=2.E+2,            # Number of function evaluations
    name='Alpine',          # Function name for log (this is optional)
    with_log=True,
    is_vect = False)


# And now we launching the maximizer:
tto.optimize(rank, is_max=True)


# We can extract the results of the computation:
x = tto.x_opt          # The found value of the maximum of the function (x)
y = tto.y_opt          # The found value of the maximum of the function (y=f(x))
k_c = tto.k_cache      # Total number of cache usage (should be 0 in this demo)
k_e = tto.k_evals      # Total number of requests to func (is always = evals)
k_t = tto.k_total      # Total number of requests (k_cache + k_evals)
t_f = tto.t_evals_mean # Average time spent to real function call for 1 point
                       # ... (see "ttopt.py" and docs for more details)


# We log the final state:
print('-' * 70 + '\n' + tto.info() +'\n\n')


Alpine-17d | evals=2.40e+01 | t_cur=4.67e+00 | y= 2.832084e+02 
Alpine-17d | evals=5.60e+01 | t_cur=1.07e+01 | y= 2.832084e+02 
Alpine-17d | evals=8.80e+01 | t_cur=1.68e+01 | y= 2.832084e+02 
Alpine-17d | evals=1.20e+02 | t_cur=2.28e+01 | y= 2.832084e+02 
Alpine-17d | evals=1.52e+02 | t_cur=2.89e+01 | y= 2.832084e+02 
Alpine-17d | evals=1.84e+02 | t_cur=3.49e+01 | y= 2.832085e+02 
Alpine-17d | evals=2.00e+02 | t_cur=3.80e+01 | y= 2.832085e+02 
----------------------------------------------------------------------
Alpine-17d | evals=2.00e+02 | t_all=3.82e+01 | y= 2.832085e+02 




In [ ]:
%timeit GP_Li.constr_matrix_melt(GP_Li.X)

In [ ]:
%timeit GP_Li.constr_matrix_melt(GP_Li.err)